In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL principal del sitio web
base_url = "https://www.allrecipes.com/"

In [4]:
# Función para extraer las categorías disponibles
def fetch_categories(base_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        page = requests.get(base_url, headers=headers)
        page.raise_for_status()
        html_content = BeautifulSoup(page.content, "html.parser")
        links = html_content.find_all("a", href=True)
        category_dict = {
            link.text.strip(): link["href"]
            for link in links
            if "/recipes/" in link["href"] and link.text.strip()
        }
        return category_dict
    except requests.exceptions.RequestException as error:
        print(f"Error al acceder a {base_url}: {error}")
        return {}

# Función para extraer recetas de una categoría
def fetch_recipes(category_link):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(category_link, headers=headers)
        response.raise_for_status()
        html_content = BeautifulSoup(response.content, "html.parser")
        recipe_elements = html_content.find_all("a", href=True)
        recipe_urls = list(set(
            element["href"]
            for element in recipe_elements
            if "/recipe/" in element["href"]
        ))
        return recipe_urls
    except requests.exceptions.RequestException as error:
        print(f"Error al procesar {category_link}: {error}")
        return []

In [5]:
# Función para extraer detalles de una receta
def extract_recipe_details(recipe_link):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        page = requests.get(recipe_link, headers=headers)
        page.raise_for_status()
        html_content = BeautifulSoup(page.content, "html.parser")

        recipe_identifier = recipe_link.split("/")[-2]
        title_meta = html_content.find("meta", {"property": "og:title"})
        recipe_title = title_meta["content"] if title_meta else "Título no disponible"

        ingredient_items = html_content.find_all("li", class_="ingredients-item")
        ingredients_list = [ingredient.get_text(strip=True) for ingredient in ingredient_items] if ingredient_items else []

        instruction_steps = html_content.find_all("li", class_="instructions-section-item")
        instructions_list = [step.get_text(strip=True) for step in instruction_steps] if instruction_steps else []

        prep_time_element = html_content.find("div", class_="recipe-meta-item-body")
        preparation_time = prep_time_element.get_text(strip=True) if prep_time_element else "Tiempo no disponible"

        return {
            "recipe_id": recipe_identifier,
            "title": recipe_title,
            "ingredients": ingredients_list,
            "instructions": instructions_list,
            "prep_time": preparation_time,
            "source_url": recipe_link
        }
    except requests.exceptions.RequestException as error:
        print(f"Error al extraer datos de {recipe_link}: {error}")
        return None

# Extraer las categorías desde el sitio
category_data = fetch_categories(base_url)

# Lista para recopilar información de todas las recetas
recipe_collection = []
limit_recipes = 500

# Recorrer las categorías y obtener recetas
for category, category_link in category_data.items():
    print(f"Analizando categoría: {category}")
    recipe_links = fetch_recipes(category_link)

    for link in recipe_links:
        recipe_info = extract_recipe_details(link)
        if recipe_info:
            recipe_collection.append(recipe_info)

        # Finalizar si se alcanza el límite
        if len(recipe_collection) >= limit_recipes:
            break
    if len(recipe_collection) >= limit_recipes:
        break

# Crear un DataFrame y exportarlo a CSV
recipes_df = pd.DataFrame(recipe_collection)
recipes_df.to_csv("recipes_dataset.csv", index=False, encoding="utf-8")
print("Archivo guardado como 'recipes_dataset.csv'.")


Analizando categoría: Dinners
Analizando categoría: 5-Ingredient Dinners
Analizando categoría: One-Pot Meals
Analizando categoría: Quick & Easy
Analizando categoría: 30-Minute Meals
Analizando categoría: Family Dinners
Analizando categoría: Soups, Stews & Chili
Analizando categoría: Comfort Food
Analizando categoría: Main Dishes
Analizando categoría: Sheet Pan Dinners
Analizando categoría: View All
Analizando categoría: Breakfast & Brunch
Analizando categoría: Lunch
Analizando categoría: Healthy
Analizando categoría: Appetizers & Snacks
Analizando categoría: Salads
Archivo guardado como 'recipes_dataset.csv'.


In [6]:
# Función para construir un corpus de recetas
def generate_recipe_corpus(category_data, recipe_limit=500):
    unique_recipes = set()
    recipe_list = []
    recipe_counter = 1

    for cat_name, cat_link in category_data.items():
        print(f"Procesando recetas de la categoría: {cat_name}")
        recipe_urls = fetch_recipes(cat_link)

        for url in recipe_urls:
            if len(unique_recipes) >= recipe_limit:
                break
            if url not in unique_recipes:
                unique_recipes.add(url)

                # Obtener el nombre de la receta desde la URL
                recipe_title = url.split("/")[-2].replace("-", " ").capitalize()
                if not recipe_title:
                    recipe_title = "Receta desconocida"

                # Agregar detalles de la receta al corpus
                recipe_list.append({
                    "recipe_id": recipe_counter,
                    "title": recipe_title,
                    "link": url
                })
                recipe_counter += 1

        if len(unique_recipes) >= recipe_limit:
            break

    return recipe_list

# Ejecución del script
if __name__ == "__main__":
    print("Extrayendo categorías del sitio web...")
    category_info = fetch_categories(base_url)

    if category_info:
        print(f"Total de categorías encontradas: {len(category_info)}")
        for cat, link in category_info.items():
            print(f"{cat}: {link}")

        print("\nIniciando la construcción del corpus de recetas...")
        recipe_corpus = generate_recipe_corpus(category_info)
        print(f"\nNúmero total de recetas en el corpus: {len(recipe_corpus)}")

        # Ejemplo: Mostrar las primeras 10 recetas
        for recipe in recipe_corpus[:10]:
            print(f"ID: {recipe['recipe_id']}, Título: {recipe['title']}, Enlace: {recipe['link']}")
    else:
        print("No se pudieron obtener categorías del sitio web.")


Extrayendo categorías del sitio web...
Total de categorías encontradas: 56
Dinners: https://www.allrecipes.com/recipes/17562/dinner/
5-Ingredient Dinners: https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
One-Pot Meals: https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
Quick & Easy: https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
30-Minute Meals: https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
Family Dinners: https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/
Soups, Stews & Chili: https://www.allrecipes.com/recipes/94/soups-stews-and-chili/
Comfort Food: https://www.allrecipes.com/recipes/16099/everyday-cooking/comfort-food/
Main Dishes: https://www.allrecipes.com/recipes/80/main-dish/
Sheet Pan Dinners: https://www.allrecipes.com/recipes/22992/everyday-cooking/sheet-pan-dinners/
View All: https://www.allrecip

In [8]:
import csv

# Función para exportar las recetas a un archivo CSV
def export_recipes_to_csv(recipe_data, output_file="recipes_dataset.csv"):
    try:
        with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.DictWriter(csvfile, fieldnames=["recipe_id", "title", "link"])
            csv_writer.writeheader()  # Agregar encabezados al archivo CSV
            csv_writer.writerows(recipe_data)  # Agregar los datos de las recetas
        print(f"Archivo CSV creado con éxito: {output_file}.")
    except Exception as error:
        print(f"Se produjo un error al intentar guardar el archivo CSV: {error}")

# Guardar los datos del corpus en un archivo CSV
export_recipes_to_csv(recipe_corpus)


Archivo CSV creado con éxito: recipes_dataset.csv.


In [9]:
# Función para leer recetas desde un archivo CSV
def import_recipes_from_csv(csv_file="recipes_dataset.csv"):
    try:
        recipe_list = []
        with open(csv_file, mode='r', encoding='utf-8') as file_handler:
            csv_reader = csv.DictReader(file_handler)
            for record in csv_reader:
                recipe_list.append(record)
        print(f"Se cargaron {len(recipe_list)} recetas desde el archivo {csv_file}.")
        return recipe_list
    except Exception as error:
        print(f"Ocurrió un problema al leer el archivo CSV: {error}")
        return []

# Leer recetas desde el archivo CSV
imported_recipes = import_recipes_from_csv()
for rec in imported_recipes[:10]:  # Mostrar las primeras 10 recetas
    print(f"ID: {rec['recipe_id']}, Título: {rec['title']}, Enlace: {rec['link']}")


Se cargaron 500 recetas desde el archivo recipes_dataset.csv.
ID: 1, Título: Easy meatloaf, Enlace: https://www.allrecipes.com/recipe/16354/easy-meatloaf/
ID: 2, Título: Corned beef and cabbage i, Enlace: https://www.allrecipes.com/recipe/16310/corned-beef-and-cabbage-i/
ID: 3, Título: Simple macaroni and cheese, Enlace: https://www.allrecipes.com/recipe/238691/simple-macaroni-and-cheese/
ID: 4, Título: Salisbury steak, Enlace: https://www.allrecipes.com/recipe/14595/salisbury-steak/
ID: 5, Título: Turkey in a smoker, Enlace: https://www.allrecipes.com/recipe/16984/turkey-in-a-smoker/
ID: 6, Título: Worlds best lasagna, Enlace: https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/
ID: 7, Título: Mississippi chicken, Enlace: https://www.allrecipes.com/recipe/8508920/mississippi-chicken/
ID: 8, Título: Chicken pot pie ix, Enlace: https://www.allrecipes.com/recipe/26317/chicken-pot-pie-ix/
ID: 9, Título: Dumplings, Enlace: https://www.allrecipes.com/recipe/6900/dumplings/
ID: 10, T

In [10]:
def extract_recipe_details(recipe_link):
    try:
        response = requests.get(recipe_link, headers=headers)
        response.raise_for_status()
        page_content = BeautifulSoup(response.content, "html.parser")

        # Extraer el título de la receta
        title = page_content.find("h1", class_="headline").text.strip() if page_content.find("h1", class_="headline") else "Título no disponible"

        # Extraer la descripción de la receta
        recipe_description = page_content.find("div", class_="intro").text.strip() if page_content.find("div", class_="intro") else "Sin descripción"

        # Extraer los ingredientes
        ingredients_list = []
        ingredient_elements = page_content.find_all("span", class_="ingredients-item-name")
        for item in ingredient_elements:
            ingredients_list.append(item.text.strip())

        # Extraer las instrucciones
        steps_list = []
        instruction_elements = page_content.find_all("li", class_="subcontainer instructions-section-item")
        for step in instruction_elements:
            steps_list.append(step.text.strip())

        # Extraer tiempo de preparación
        preparation_time = page_content.find("span", class_="prepTime__item--time").text.strip() if page_content.find("span", class_="prepTime__item--time") else "Tiempo no disponible"

        # Devolver un diccionario con los datos de la receta
        return {
            "title": title,
            "link": recipe_link,
            "description": recipe_description,
            "ingredients": ingredients_list,
            "steps": steps_list,
            "prep_time": preparation_time
        }

    except requests.exceptions.RequestException as error:
        print(f"Error al procesar {recipe_link}: {error}")
        return {}


In [15]:
def create_recipe_corpus(category_data, recipe_limit=500):
    unique_recipes = set()
    corpus_data = []
    identifier = 1

    for category, category_link in category_data.items():
        print(f"Procesando recetas de la categoría: {category}")
        recipe_links = fetch_recipes(category_link)

        for link in recipe_links:
            if len(unique_recipes) >= recipe_limit:
                break
            if link not in unique_recipes:
                unique_recipes.add(link)

                # Extraer detalles de la receta
                details = extract_recipe_details(link)
                if details:  # Verificar que los detalles no estén vacíos
                    corpus_data.append({
                        "recipe_id": identifier,
                        "title": details["title"],
                        "link": details["link"],
                        "description": details["description"],
                        "ingredients": details["ingredients"],
                        "steps": details["steps"],
                        "prep_time": details["prep_time"],
                        "category": category
                    })
                    identifier += 1

        if len(unique_recipes) >= recipe_limit:
            break

    return corpus_data


In [22]:
import csv
from google.colab import files

def export_complete_recipes_to_csv(data, output_file="recipes_full_data.csv"):
    try:
        with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
            headers = ["recipe_id", "title", "category", "link", "description", "ingredients", "steps", "prep_time"]
            csv_writer = csv.DictWriter(csvfile, fieldnames=headers)
            csv_writer.writeheader()  # Escribir los encabezados en el archivo CSV

            for item in data:
                # Convertir listas de ingredientes e instrucciones en cadenas de texto, manejando valores nulos
                item["ingredients"] = ", ".join(str(ingredient) for ingredient in item.get("ingredients", []) if ingredient)
                item["steps"] = " | ".join(str(step) for step in item.get("steps", []) if step)
                csv_writer.writerow(item)  # Escribir cada registro al archivo
        print(f"Archivo de recetas exportado correctamente a {output_file}.")
    except Exception as error:
        print(f"Se produjo un error al guardar el archivo CSV: {error}")

# Descargar el archivo CSV generado
files.download("recipes_full_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
# Función para leer recetas completas desde un archivo CSV
def import_complete_recipes_from_csv(csv_file="recipes_full_data.csv"):
    try:
        recipe_data = []
        with open(csv_file, mode='r', encoding='utf-8') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            for record in csv_reader:
                recipe_data.append(record)
        print(f"Se cargaron {len(recipe_data)} recetas completas desde {csv_file}.")
        return recipe_data
    except Exception as error:
        print(f"Ocurrió un error al cargar el archivo CSV: {error}")
        return []

from google.colab import files

# Guardar y descargar el archivo CSV
export_complete_recipes_to_csv(recipe_corpus)

# Descargar el archivo generado
files.download("recipes_full_data.csv")

print(f"Cantidad total de recetas obtenidas: {len(recipe_corpus)}")
if recipe_corpus:
    export_complete_recipes_to_csv(recipe_corpus)
else:
    print("No se encontraron recetas para guardar.")

# Imprimir las primeras 5 recetas para verificar el contenido
print(recipe_corpus[:5])


Archivo de recetas exportado correctamente a recipes_full_data.csv.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cantidad total de recetas obtenidas: 500
Archivo de recetas exportado correctamente a recipes_full_data.csv.
[{'recipe_id': 1, 'title': 'Easy meatloaf', 'link': 'https://www.allrecipes.com/recipe/16354/easy-meatloaf/', 'ingredients': '', 'steps': ''}, {'recipe_id': 2, 'title': 'Corned beef and cabbage i', 'link': 'https://www.allrecipes.com/recipe/16310/corned-beef-and-cabbage-i/', 'ingredients': '', 'steps': ''}, {'recipe_id': 3, 'title': 'Simple macaroni and cheese', 'link': 'https://www.allrecipes.com/recipe/238691/simple-macaroni-and-cheese/', 'ingredients': '', 'steps': ''}, {'recipe_id': 4, 'title': 'Salisbury steak', 'link': 'https://www.allrecipes.com/recipe/14595/salisbury-steak/', 'ingredients': '', 'steps': ''}, {'recipe_id': 5, 'title': 'Turkey in a smoker', 'link': 'https://www.allrecipes.com/recipe/16984/turkey-in-a-smoker/', 'ingredients': '', 'steps': ''}]
